# Delta Lake Table Cloning and Time Travel
## An Interactive Education Demo for Healthcare Payers

This notebook demonstrates Delta Lake's time travel, shallow clone, and deep clone features using PySpark SQL with managed tables only. 

**Story**: HealthFirst Insurance managing member claims data with compliance, dev/test, and disaster recovery needs.


---

## 📋 Table of Contents
1. [Core Concepts](#core-concepts)
2. [Configuration](#configuration)
3. [Demo: Time Travel, Cloning & Recovery](#demo)
4. [Real-World Healthcare Scenarios](#scenarios)
5. [Best Practices & Key Takeaways](#best-practices)
6. [Additional Resources](#resources)

---

## 🎯 Core Concepts

### What is Time Travel?
Delta Lake's **time travel** allows you to query, restore, or audit previous versions of your data. Every modification creates a new version—perfect for:
- **Compliance audits**: "Show me the member claims as they existed on January 31st"
- **Rollback mistakes**: Undo accidental updates or deletes
- **Data lineage**: Track how member status changed over time

### What is a Shallow Clone?
A **shallow clone** creates a new table that references the same underlying data files as the source table—no data is copied initially.

**Healthcare Use Case**: Create a `member_claims_prod_dev` table for developers to test new adjudication logic without duplicating 500GB of production claims data.

**Key Characteristic**: Metadata-only operation (fast and cheap), but dependent on source table.

### What is a Deep Clone?
A **deep clone** creates a completely independent copy of the table, including all data files.

**Healthcare Use Case**: Create `member_claims_backup` for disaster recovery before a major migration or system upgrade.

**Key Characteristic**: Full data copy (slower and more storage), but completely independent.

---

## 📊 Shallow Clone vs Deep Clone: Quick Comparison

| Feature | **Shallow Clone** | **Deep Clone** |
|---------|-------------------|----------------|
| **Data Copied?** | ❌ No (references same files) | ✅ Yes (full copy) |
| **Speed** | ⚡ Fast (metadata only) | 🐢 Slower (copies all data) |
| **Storage Cost** | 💰 Low (minimal) | 💰💰 Higher (full duplication) |
| **Independence** | ⚠️ Depends on source table | ✅ Fully independent |
| **Use After Source DROP** | ⚠️ Queryable but "broken" for 7 days* | ✅ Works perfectly |
| **Best For** | Dev/test, analytics sandboxes | DR, migration, archival |

*Unity Catalog's 7-day retention allows queries but blocks metadata operations like VACUUM until source is restored.

---

## Configuration

Update these values for your environment:


---

## 🏥 Real-World Healthcare Payer Scenarios

### 1. **Compliance Audit**
**Challenge**: CMS requests claims data "as it existed on March 31st for audit"  
**Solution**: Use time travel to query historical version
```sql
SELECT * FROM member_claims_prod TIMESTAMP AS OF '2025-03-31T23:59:59Z';
```

### 2. **Testing Claim Adjudication Rules**
**Challenge**: New auto-adjudication logic needs testing without risking production data  
**Solution**: Shallow clone production to dev environment
```sql
CREATE TABLE member_claims_dev SHALLOW CLONE member_claims_prod;
-- Test new logic on shallow clone
```

### 3. **Pre-Migration Backup**
**Challenge**: Migrating to new billing system; need 100% guaranteed backup  
**Solution**: Deep clone before migration
```sql
CREATE TABLE member_claims_backup_2025_Q1 DEEP CLONE member_claims_prod;
```

### 4. **Accidental Data Deletion**
**Challenge**: Developer accidentally runs `DELETE` on production table  
**Solution**: Restore from previous version using time travel
```sql
RESTORE TABLE member_claims_prod TO VERSION AS OF 42;
```

### 5. **Multi-Region Disaster Recovery**
**Challenge**: Need claims data available in secondary region for DR  
**Solution**: Deep clone to DR region (with appropriate `LOCATION` for external tables)

---


In [38]:
# Configuration
catalog_name = "quickstart_catalog_vkm_external"
schema_name = "claims_analytics"
table_name = "member_claims_prod"

spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

print(f"Using: {catalog_name}.{schema_name}.{table_name}")


Using: quickstart_catalog_vkm_external.claims_analytics.member_claims_prod


---

## 🏥 Demo: Time Travel, Cloning & Recovery

### Healthcare Payer Storyline

**You are a Data Engineer at HealthFirst Insurance**, managing member claims data across multiple environments:

- **Production Table** (`member_claims_prod`): Live claims data powering billing, reporting, and compliance
- **Dev/Test Environment**: Developers need to test new claim adjudication rules without affecting production
- **Disaster Recovery**: Compliance requires backups before major system changes
- **Audit Requirements**: Regulators may ask "What was the claim status on February 15th?"

This demo walks through a realistic workflow using Delta Lake's time travel and cloning features.

---

## Step 1: Create Production Claims Table


In [39]:
# Create managed table
spark.sql(f"""
CREATE OR REPLACE TABLE {table_name} (
  claim_id STRING,
  member_id STRING,
  provider_npi STRING,
  diagnosis_code STRING,
  claim_amount DECIMAL(10,2),
  service_date DATE,
  adjudication_status STRING,
  last_updated TIMESTAMP
) USING DELTA
""")
print(f"✅ Created: {table_name}")


✅ Created: member_claims_prod


In [40]:
# Insert data (Version 1)
spark.sql(f"""
INSERT INTO {table_name} VALUES
  ('CLM-001', 'MEM-78451', '1234567890', 'E11.9', 1250.00, '2025-01-15', 'PENDING', current_timestamp()),
  ('CLM-002', 'MEM-78452', '1234567891', 'I10', 450.75, '2025-01-16', 'APPROVED', current_timestamp()),
  ('CLM-003', 'MEM-78453', '1234567892', 'J44.0', 890.25, '2025-01-17', 'PENDING', current_timestamp()),
  ('CLM-004', 'MEM-78454', '1234567893', 'M79.3', 325.50, '2025-01-18', 'APPROVED', current_timestamp()),
  ('CLM-005', 'MEM-78455', '1234567894', 'F41.1', 675.00, '2025-01-19', 'PENDING', current_timestamp())
""")
display(spark.sql(f"SELECT * FROM {table_name} ORDER BY claim_id"))


,claim_id,member_id,provider_npi,diagnosis_code,claim_amount,service_date,adjudication_status,last_updated
0,CLM-001,MEM-78451,1234567890,E11.9,1250.00,2025-01-15,PENDING,2025-11-05 17:28:52.219467
1,CLM-002,MEM-78452,1234567891,I10,450.75,2025-01-16,APPROVED,2025-11-05 17:28:52.219467
2,CLM-003,MEM-78453,1234567892,J44.0,890.25,2025-01-17,PENDING,2025-11-05 17:28:52.219467
3,CLM-004,MEM-78454,1234567893,M79.3,325.50,2025-01-18,APPROVED,2025-11-05 17:28:52.219467
4,CLM-005,MEM-78455,1234567894,F41.1,675.00,2025-01-19,PENDING,2025-11-05 17:28:52.219467


## Step 2: Create Version History


In [41]:
# Update (Version 2) + Insert (Version 3)
spark.sql(f"UPDATE {table_name} SET adjudication_status = 'APPROVED' WHERE adjudication_status = 'PENDING'")
spark.sql(f"""
INSERT INTO {table_name} VALUES
  ('CLM-006', 'MEM-78456', '1234567895', 'K21.9', 550.00, '2025-01-20', 'PENDING', current_timestamp()),
  ('CLM-007', 'MEM-78457', '1234567896', 'N18.3', 1875.50, '2025-01-21', 'UNDER_REVIEW', current_timestamp()),
  ('CLM-008', 'MEM-78458', '1234567897', 'E78.5', 290.00, '2025-01-22', 'APPROVED', current_timestamp())
""")
print("✅ Created 3 versions")


✅ Created 3 versions


## Step 3: Time Travel


In [42]:
# Time travel - query Version 1 (first INSERT)
display(spark.sql(f"DESCRIBE HISTORY {table_name}"))
print("\nVersion 1 - Original data:")
display(spark.sql(f"SELECT * FROM {table_name} VERSION AS OF 1 ORDER BY claim_id"))


,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,4,2025-11-05 17:29:02,8832166061032092,vik.malhotra@databricks.com,OPTIMIZE,"{'clusterBy': '[]', 'zOrderBy': '[]', 'batchId': '0', 'predicate': '[]', 'auto': 'true'}",None,None,1105-171416-wzji5xnm-v2n,2.0,SnapshotIsolation,False,"{'numRemovedFiles': '2', 'numRemovedBytes': '5684', 'p25FileSize': '3342', 'numDeletionVectorsRemoved': '1', 'conflictDetectionTimeMs': '88', 'minFileSize': '3342', 'p75FileSize': '3342', 'p50FileSize': '3342', 'numAddedBytes': '3342', 'numAddedFiles': '1', 'maxFileSize': '3342'}",None,Databricks-Runtime/17.2.x-photon-scala2.13
1,3,2025-11-05 17:29:00,8832166061032092,vik.malhotra@databricks.com,WRITE,"{'mode': 'Append', 'statsOnLoad': 'false', 'partitionBy': '[]'}",None,None,1105-171416-wzji5xnm-v2n,2.0,WriteSerializable,True,"{'numFiles': '1', 'numOutputRows': '3', 'numOutputBytes': '2431'}",None,Databricks-Runtime/17.2.x-photon-scala2.13
2,2,2025-11-05 17:28:58,8832166061032092,vik.malhotra@databricks.com,UPDATE,"{'predicate': '[""(adjudication_status#17053 = PENDING)""]'}",None,None,1105-171416-wzji5xnm-v2n,1.0,WriteSerializable,False,"{'numRemovedFiles': '0', 'numRemovedBytes': '0', 'numCopiedRows': '0', 'numDeletionVectorsAdded': '1', 'executionTimeMs': '2018', 'numDeletionVectorsUpdated': '0', 'scanTimeMs': '802', 'numAddedFiles': '1', 'numUpdatedRows': '3', 'numDeletionVectorsRemoved': '0', 'numAddedChangeFiles': '0', 'numAddedBytes': '3197', 'rewriteTimeMs': '1215'}",None,Databricks-Runtime/17.2.x-photon-scala2.13
3,1,2025-11-05 17:28:53,8832166061032092,vik.malhotra@databricks.com,WRITE,"{'mode': 'Append', 'statsOnLoad': 'false', 'partitionBy': '[]'}",None,None,1105-171416-wzji5xnm-v2n,0.0,WriteSerializable,True,"{'numFiles': '1', 'numOutputRows': '5', 'numOutputBytes': '2487'}",None,Databricks-Runtime/17.2.x-photon-scala2.13
4,0,2025-11-05 17:28:51,8832166061032092,vik.malhotra@databricks.com,CREATE OR REPLACE TABLE,"{'partitionBy': '[]', 'clusterBy': '[]', 'description': None, 'isManaged': 'true', 'properties': '{""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true"",""delta.enableRowTracking"":""true"",""delta.rowTracking.materializedRowCommitVersionColumnName"":""_row-commit-version-col-16526301-bd61-4b6f-b78d-ae2e65bc6495"",""delta.rowTracking.materializedRowIdColumnName"":""_row-id-col-213e9716-37fc-4dff-a7c4-7b37db6cb507""}', 'statsOnLoad': 'false'}",None,None,1105-171416-wzji5xnm-v2n,NaN,WriteSerializable,True,{},None,Databricks-Runtime/17.2.x-photon-scala2.13



Version 1 - Original data:


,claim_id,member_id,provider_npi,diagnosis_code,claim_amount,service_date,adjudication_status,last_updated
0,CLM-001,MEM-78451,1234567890,E11.9,1250.00,2025-01-15,PENDING,2025-11-05 17:28:52.219467
1,CLM-002,MEM-78452,1234567891,I10,450.75,2025-01-16,APPROVED,2025-11-05 17:28:52.219467
2,CLM-003,MEM-78453,1234567892,J44.0,890.25,2025-01-17,PENDING,2025-11-05 17:28:52.219467
3,CLM-004,MEM-78454,1234567893,M79.3,325.50,2025-01-18,APPROVED,2025-11-05 17:28:52.219467
4,CLM-005,MEM-78455,1234567894,F41.1,675.00,2025-01-19,PENDING,2025-11-05 17:28:52.219467


## Step 4: Shallow Clone (Dev/Test Environment)

**Scenario**: Your development team needs to test new claim processing logic. Instead of copying 500GB of production data, create a shallow clone that references the same data files.

**Benefits**:
- ⚡ Instant creation (metadata only)
- 💰 No additional storage cost initially
- 🔒 Schema changes in dev won't affect production
- 📊 Developers work with real production data structure


In [43]:
# Shallow clone
dev_table = f"{table_name}_dev"
spark.sql(f"DROP TABLE IF EXISTS {dev_table}")
spark.sql(f"CREATE TABLE {dev_table} SHALLOW CLONE {table_name}")
print(f"✅ Created shallow clone: {dev_table}")


✅ Created shallow clone: member_claims_prod_dev


## Step 5: Deep Clone (Disaster Recovery)

**Scenario**: Before a major system upgrade (e.g., migrating to a new claims adjudication system), compliance requires a full backup that will remain accessible even if production is deleted.

**Benefits**:
- ✅ Completely independent copy
- 🛡️ Survives source table deletion
- 📦 Can be stored in different regions/accounts
- 🔄 Perfect for disaster recovery and audits


In [44]:
# Deep clone  
backup_table = f"{table_name}_backup"
spark.sql(f"DROP TABLE IF EXISTS {backup_table}")
spark.sql(f"CREATE TABLE {backup_table} DEEP CLONE {table_name}")
print(f"✅ Created deep clone: {backup_table}")


✅ Created deep clone: member_claims_prod_backup


## ⚠️ Step 6: DROP Test (Shallow vs Deep Clone)

**This demonstrates Unity Catalog's 7-day safety net:**
- Shallow clone: Queryable but "broken" for metadata operations
- Deep clone: Completely unaffected


In [45]:
# DROP source table
print(f"⚠️ Dropping {table_name}...")
spark.sql(f"DROP TABLE {table_name}")

# Test clones
try:
    count_dev = spark.sql(f"SELECT COUNT(*) as count FROM {dev_table}").collect()[0]['count']
    print(f"✅ Shallow clone: {count_dev} rows (queryable but broken for VACUUM)")
except Exception as e:
    print(f"❌ Shallow clone failed: {e}")

try:
    count_backup = spark.sql(f"SELECT COUNT(*) as count FROM {backup_table}").collect()[0]['count']
    print(f"✅ Deep clone: {count_backup} rows (completely independent!)")
except Exception as e:
    print(f"❌ Deep clone failed: {e}")


⚠️ Dropping member_claims_prod...
✅ Shallow clone: 8 rows (queryable but broken for VACUUM)
✅ Deep clone: 8 rows (completely independent!)


## Step 7: UNDROP - Recover Table


In [46]:
# UNDROP recovers table within 7-day window
print(f"🔄 Using UNDROP to recover {table_name}...")
try:
    spark.sql(f"UNDROP TABLE {table_name}")
    count = spark.sql(f"SELECT COUNT(*) FROM {table_name}").collect()[0][0]
    print(f"✅ Table recovered! {count} rows restored")
    print(f"✅ Shallow clone now fully functional again")
except Exception as e:
    print(f"❌ UNDROP failed: {e}")


🔄 Using UNDROP to recover member_claims_prod...
✅ Table recovered! 8 rows restored
✅ Shallow clone now fully functional again


## Step 8: VACUUM


In [47]:
# VACUUM (after UNDROP)
print(f"🧹 VACUUM {dev_table}...")
spark.sql(f"VACUUM {dev_table} RETAIN 168 HOURS")
print("✅ VACUUM completed")


🧹 VACUUM member_claims_prod_dev...
✅ VACUUM completed


## Cleanup


---

## 🎯 Key Takeaways & Best Practices

### When to Use Each Feature

| Feature | Best Use Cases | Avoid When |
|---------|---------------|------------|
| **Time Travel** | Audits, rollbacks, debugging | Extremely old versions (check retention) |
| **Shallow Clone** | Dev/test, analytics sandboxes | Need guaranteed independence from source |
| **Deep Clone** | DR, migration, archival, cross-region | Budget-constrained (high storage cost) |

### Best Practices for Healthcare Payers

1. **🔒 Shallow Clones for Development**
   - Use shallow clones for dev/test environments to save storage costs
   - Remember: Dependent on source table—don't drop production without deep clone backup!

2. **🛡️ Deep Clones for Compliance**
   - Create deep clones before major system changes (migrations, upgrades)
   - Store quarterly deep clones for long-term audit requirements
   - Consider cross-region deep clones for disaster recovery

3. **📅 Time Travel for Audits**
   - Leverage `VERSION AS OF` or `TIMESTAMP AS OF` for regulatory audits
   - Set appropriate `delta.logRetentionDuration` (default 30 days) based on compliance needs
   - Document version numbers for critical data snapshots

4. **🧹 VACUUM with Caution**
   - `VACUUM` removes old data files—disables time travel to those versions
   - Default retention: 7 days (168 hours)
   - Never run `VACUUM RETAIN 0 HOURS` on production without backups

5. **🔄 UNDROP as Safety Net**
   - Unity Catalog retains dropped tables for 7 days
   - Use `UNDROP TABLE` to recover accidentally dropped tables
   - Shallow clones remain queryable during this period but "broken" for metadata ops

### Common Pitfalls to Avoid

❌ **Don't** drop source tables with active shallow clones in production  
❌ **Don't** assume shallow clones survive source deletion (they're queryable but "broken")  
❌ **Don't** forget to deep clone before major migrations  
❌ **Don't** run VACUUM too aggressively (kills time travel history)

✅ **Do** use shallow clones for cost-effective dev/test  
✅ **Do** use deep clones for disaster recovery  
✅ **Do** document version numbers for compliance snapshots  
✅ **Do** test UNDROP and backup procedures regularly

---


---

## 📚 Additional Resources

### Official Azure Databricks Documentation

#### Time Travel
- [**Time Travel in Delta Lake Tables**](https://learn.microsoft.com/en-us/azure/databricks/delta/time-travel)  
  Query and restore previous versions of your Delta tables

- [**RESTORE TABLE Command**](https://docs.databricks.com/sql/language-manual/delta-restore.html)  
  Restore a Delta table to an earlier version

#### Cloning
- [**Clone a Table on Azure Databricks**](https://learn.microsoft.com/en-us/azure/databricks/delta/clone)  
  Comprehensive guide to deep and shallow clones

- [**Shallow Clone for Unity Catalog Tables**](https://learn.microsoft.com/en-us/azure/databricks/delta/clone-unity-catalog)  
  Unity Catalog-specific cloning behavior and limitations

- [**CREATE TABLE CLONE SQL Syntax**](https://docs.databricks.com/sql/language-manual/delta-clone.html)  
  Full SQL reference for CLONE commands

#### Table Management
- [**DROP TABLE Command**](https://docs.databricks.com/sql/language-manual/sql-ref-syntax-ddl-drop-table.html)  
  DROP TABLE syntax and Unity Catalog retention policies

- [**UNDROP TABLE Command**](https://docs.databricks.com/sql/language-manual/sql-ref-syntax-ddl-undrop-table.html)  
  Recover dropped tables within 7-day retention window

- [**VACUUM Command**](https://learn.microsoft.com/en-us/azure/databricks/delta/vacuum)  
  Remove unused data files to reclaim storage

#### Unity Catalog
- [**Unity Catalog Best Practices**](https://learn.microsoft.com/en-us/azure/databricks/data-governance/unity-catalog/best-practices)  
  Governance, security, and management recommendations

- [**Managed Tables vs External Tables**](https://docs.databricks.com/sql/language-manual/sql-ref-syntax-ddl-create-table.html)  
  Understanding table types in Unity Catalog

### Delta Lake Resources
- [**Delta Lake Official Documentation**](https://docs.delta.io/)  
  Core Delta Lake concepts and features

- [**Delta Lake Transaction Log**](https://docs.delta.io/latest/delta-intro.html#transaction-log)  
  How Delta Lake tracks versions and enables time travel

---

## 🎉 Congratulations!

You've successfully learned how to:
- ✅ Use **time travel** to query and audit historical data
- ✅ Create **shallow clones** for cost-effective dev/test environments
- ✅ Create **deep clones** for disaster recovery and compliance
- ✅ Understand **Unity Catalog's 7-day retention** for dropped tables
- ✅ Use **UNDROP** to recover accidentally deleted tables
- ✅ Apply **VACUUM** to reclaim storage while preserving history

### Next Steps
1. Try this notebook in your own Databricks workspace
2. Experiment with `RESTORE TABLE` for rollback scenarios
3. Set up automated deep clones for critical production tables
4. Configure `delta.logRetentionDuration` for your compliance needs

**Questions?** Refer to the official documentation links above or reach out to your Databricks account team.

---

*This notebook demonstrates managed tables in Unity Catalog. For external tables with custom storage locations, consult the Databricks documentation for `LOCATION` clause usage.*


In [48]:
# Cleanup all demo tables
for tbl in [table_name, dev_table, backup_table]:
    try:
        spark.sql(f"DROP TABLE IF EXISTS {tbl}")
        print(f"✅ Dropped: {tbl}")
    except:
        pass
print("\n✅ Cleanup complete!")


✅ Dropped: member_claims_prod
✅ Dropped: member_claims_prod_dev
✅ Dropped: member_claims_prod_backup

✅ Cleanup complete!
